In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_4/models/B5R2b5_AN_1FC_fold4_1.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold4.csv') #เปลี่ยน Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(4599, 25)
(4599, 25)
Normal:  (3004, 25)
Abnormal:  (1595, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,0,0,15,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,35.0,96.0,694.0,584.0,0.036832,0.107041,0.712707,0.649402,AB01 P3-1 C015.JPG,AB01 P3-1 C015.JPG
1,1,1,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267,AB01 P1 C015.JPG,AB01 P1 C015.JPG
2,2,2,2,2,15,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,242.0,101.0,491.0,451.0,0.245009,0.113017,0.497278,0.501992,AB01 P2 C015.JPG,AB01 P2 C015.JPG
3,3,3,3,3,15,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,154.0,110.0,714.0,611.0,0.155235,0.122977,0.718412,0.679283,AB01 P4-2 C015.JPG,AB01 P4-2 C015.JPG
4,4,4,4,4,15,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,152.0,146.0,609.0,519.0,0.152057,0.162817,0.608229,0.577689,AB01 P4-1 C015.JPG,AB01 P4-1 C015.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 4599 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.15702996, -0.12617734,  0.28387541, ..., -0.0103116 ,
         0.13094233, -0.1140648 ],
       [-0.05845842, -0.14123163,  0.36276752, ...,  0.07865838,
         0.26696232, -0.13735317],
       [-0.08137225, -0.15796874,  0.14336134, ..., -0.08298479,
         0.05163857, -0.0743714 ],
       ...,
       [ 0.68898195,  0.55751979, -0.10279335, ...,  0.63315755,
        -0.14189151,  0.75788844],
       [ 0.19503953,  0.23783135, -0.07735135, ...,  1.06719995,
        -0.20991261,  1.34888995],
       [ 0.10759547,  0.74349821, -0.19759367, ...,  0.82685626,
        -0.21626598,  0.45564482]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.157030,-0.126177,0.283875,0.177955,-0.204914,-0.119179,0.177589,-0.202635,-0.202246,-0.142111,...,-0.137950,-0.045568,-0.008590,-0.147743,-0.119372,0.099901,-0.084402,-0.010312,0.130942,-0.114065
1,-0.058458,-0.141232,0.362768,0.277440,-0.200999,-0.037659,0.485748,-0.209703,-0.177029,-0.142544,...,-0.115953,0.078149,0.079545,-0.019242,-0.188589,0.100490,0.052707,0.078658,0.266962,-0.137353
2,-0.081372,-0.157969,0.143361,0.329052,-0.167163,-0.073277,0.483238,-0.221646,-0.199772,-0.195145,...,-0.111053,0.177609,0.049111,0.016105,-0.200784,0.020935,0.001936,-0.082985,0.051639,-0.074371
3,-0.174167,-0.159779,0.162509,0.050654,-0.206839,-0.075980,0.234413,-0.118046,-0.227471,-0.179677,...,-0.112801,-0.016988,-0.041803,-0.099391,-0.202915,0.133779,-0.083848,-0.111773,0.106413,-0.144696
4,0.035648,-0.125566,0.521049,0.137094,-0.069219,-0.107364,0.380141,-0.126261,-0.154846,-0.106276,...,-0.076898,-0.035141,-0.027952,0.061169,-0.063810,0.046267,-0.031627,-0.072788,0.157180,-0.146621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4594,1.151445,0.996131,-0.159156,-0.140687,0.646209,-0.154899,-0.137270,0.413653,0.749244,0.594663,...,0.406329,-0.198439,-0.126801,0.985542,0.883974,-0.213155,-0.195818,0.877931,-0.211737,0.939114
4595,0.648764,0.264485,-0.140569,-0.068871,0.618294,-0.140004,-0.144066,0.692879,0.171099,0.420510,...,0.469703,-0.112030,0.037296,0.813257,0.614947,-0.172064,-0.080685,0.358780,-0.042262,0.407715
4596,0.688982,0.557520,-0.102793,-0.134949,0.561530,-0.022127,-0.075646,0.281109,0.445180,0.285401,...,0.726367,-0.131213,-0.035962,0.708831,0.671722,-0.202737,-0.169453,0.633158,-0.141892,0.757888
4597,0.195040,0.237831,-0.077351,-0.140860,0.829676,-0.207316,-0.207581,0.463193,0.460921,0.955737,...,0.629336,-0.182578,-0.205578,0.556097,0.371036,-0.216708,-0.151701,1.067200,-0.209913,1.348890


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.157030,-0.126177,0.283875,0.177955,-0.204914,-0.119179,0.177589,-0.202635,-0.202246,-0.142111,...,-0.119372,0.099901,-0.084402,-0.010312,0.130942,-0.114065,Abnormal,P31,FP-B,AB01
1,-0.058458,-0.141232,0.362768,0.277440,-0.200999,-0.037659,0.485748,-0.209703,-0.177029,-0.142544,...,-0.188589,0.100490,0.052707,0.078658,0.266962,-0.137353,Abnormal,P1,FP-A,AB01
2,-0.081372,-0.157969,0.143361,0.329052,-0.167163,-0.073277,0.483238,-0.221646,-0.199772,-0.195145,...,-0.200784,0.020935,0.001936,-0.082985,0.051639,-0.074371,Abnormal,P2,FP-A,AB01
3,-0.174167,-0.159779,0.162509,0.050654,-0.206839,-0.075980,0.234413,-0.118046,-0.227471,-0.179677,...,-0.202915,0.133779,-0.083848,-0.111773,0.106413,-0.144696,Abnormal,P42,FP-B,AB01
4,0.035648,-0.125566,0.521049,0.137094,-0.069219,-0.107364,0.380141,-0.126261,-0.154846,-0.106276,...,-0.063810,0.046267,-0.031627,-0.072788,0.157180,-0.146621,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4594,1.151445,0.996131,-0.159156,-0.140687,0.646209,-0.154899,-0.137270,0.413653,0.749244,0.594663,...,0.883974,-0.213155,-0.195818,0.877931,-0.211737,0.939114,Normal,P32,FP-B,Normal
4595,0.648764,0.264485,-0.140569,-0.068871,0.618294,-0.140004,-0.144066,0.692879,0.171099,0.420510,...,0.614947,-0.172064,-0.080685,0.358780,-0.042262,0.407715,Normal,P42,FP-B,Normal
4596,0.688982,0.557520,-0.102793,-0.134949,0.561530,-0.022127,-0.075646,0.281109,0.445180,0.285401,...,0.671722,-0.202737,-0.169453,0.633158,-0.141892,0.757888,Normal,P52,FP-C,Normal
4597,0.195040,0.237831,-0.077351,-0.140860,0.829676,-0.207316,-0.207581,0.463193,0.460921,0.955737,...,0.371036,-0.216708,-0.151701,1.067200,-0.209913,1.348890,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_AN_RF_AN_fold4_1.csv') #เปลี่ยนชื่อไฟล์